In [ ]:
# 8-Modify with Edge Trigger
import cv2
import numpy as np
from collections import deque
#cap = cv2.VideoCapture(".\image\Car_FangResize.mp4")
cap = cv2.VideoCapture(".\image\DIE24.mp4")
#cap = cv2.VideoCapture(".\image\Chili.mp4")
object_detector = cv2.createBackgroundSubtractorMOG2()

ret, frame = cap.read()
height, width = 1000,2000
referenceLine =600 # 90%
ImageWidth = width
#print(height,width)

min_threshold = 10                      # these values are used to filter our detector.
max_threshold = 200                     # they can be tweaked depending on the camera distance, camera angle, ...
min_area = 100                         # ... focus, brightness, etc.
min_circularity = 0.3
min_inertia_ratio = 0.5


positionText = (50,300)
positionTextL=(100,600)

nCar = 0
BoxInline_y = 1
BoxInline_x = 1


#fourcc = cv2.VideoWriter_fourcc(* 'XVID')
#out = cv2.VideoWriter('./images/saveme.avi',fourcc,25,(1280,720))

counter = 0                           # script will use a counter to handle FPS.
readings = deque([0, 0], maxlen=10)     # lists are used to track the number of pips.
display = deque([0, 0], maxlen=10)



while(cap.isOpened()):
    ret, frame = cap.read()
    roi = frame[300:675,400: 1000]
    params = cv2.SimpleBlobDetector_Params()                # declare filter parameters.
    params.filterByArea = True
    params.filterByCircularity = True
    params.filterByInertia = True
    params.minThreshold = min_threshold
    params.maxThreshold = max_threshold
    params.minArea = min_area
    params.minCircularity = min_circularity
    params.minInertiaRatio = min_inertia_ratio
    #roi = ret[300:675,400: 1000]
    
    
    
    
    detector = cv2.SimpleBlobDetector_create(params)        # create a blob detector object.

    keypoints = detector.detect(roi)                         # keypoints is a list containing the detected blobs.
    
    # here we draw keypoints on the frame.
    
    if counter % 10 == 0:                                   # enter this block every 10 frames.
        reading = len(keypoints)                            # 'reading' counts the number of keypoints (pips).
        readings.append(reading)                            # record the reading from this frame.
 
    if readings[-1] == readings[-2] == readings[-3]:    # if the last 3 readings are the same...
            display.append(readings[-1])                    # ... then we have a valid reading.
 
        # if the most recent valid reading has changed, and it's something other than zero, then print it.
    if display[-1] != display[-2] and display[-1] != 0:
        msg = f"{display[-1]}\n"
        print(msg)
 
    counter += 1
    msg = f"{display[-1]}\n"
    im_with_keypoints = cv2.drawKeypoints(roi, keypoints, np.array([]), (0, 0, 255),
                                          cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    cv2.putText(frame,msg,positionText, cv2.FONT_HERSHEY_PLAIN, 4, (255, 0, 0), 2)  







    if frame is None:
        break

    BoxInline_y = BoxInline_x
    BoxInline_x = 0   
    
    mask = object_detector.apply(frame)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        # Calculate area and remove small elements
        area = cv2.contourArea(cnt)
        if area > 5000:
            cv2.drawContours(frame, [cnt], -1, (0, 255, 0), 2)
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 3)
            
            if y < referenceLine and (y+h) > referenceLine:
                BoxInline_x = 1    
    
    if BoxInline_y==0 and BoxInline_x==1:
        nCar = nCar + 1
    
    
    textShow = str(nCar)+'='+str(BoxInline_x)+','+str(BoxInline_y)         
    cv2.line(frame,(0,referenceLine),(ImageWidth,referenceLine),(255, 0, 0), 3)
    cv2.putText(frame,textShow , positionTextL, cv2.FONT_HERSHEY_PLAIN, 4, (255, 0, 0), 2) 
    cv2.imshow("Dice Reader", im_with_keypoints) 
    cv2.imshow("Mask", mask)
    cv2.imshow("Frame", frame)
   # cv2.imshow("roi",roi)
    if cv2.waitKey(1) & 0xff == 27:                          # press [Esc] to exit.
        break
 

 
cv2.destroyAllWindows()

1

2

1

3

4

2

5

3

5

6



IndentationError: unexpected indent (3655181486.py, line 62)